<a href="https://colab.research.google.com/github/jeetu-g2-1/Stock-invest-ai-agent/blob/main/varient_user_preferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

def get_company_data(symbol):
    api_key = '1HRCG3M2TLBM8PIS'
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    return response.json()

def clean_value(value):
    try:
        return float(value)
    except (TypeError, ValueError):
        return 0.0

def heuristic_score(company, profile):
    market_cap = clean_value(company.get('MarketCapitalization'))
    pe_ratio = clean_value(company.get('PERatio'))
    dividend_yield = clean_value(company.get('DividendYield'))
    earnings_growth = clean_value(company.get('EPS')) / max(clean_value(company.get('PreviousClose')), 1)

    if profile['risk_tolerance'] == 'conservative':
        score = (dividend_yield + 1 / (pe_ratio + 1)) * 2
    elif profile['risk_tolerance'] == 'moderate':
        score = (dividend_yield + earnings_growth) / (pe_ratio + 1)
    elif profile['risk_tolerance'] == 'aggressive':
        score = earnings_growth
    else:
        score = 0

    if profile['investment_goals'] == 'long_term_growth':
        score *= earnings_growth
    elif profile['investment_goals'] == 'short_term_income':
        score *= dividend_yield

    return score

def recommend_investment(amount, companies_data, profile):
    investment_recommendations = {}
    for company in companies_data:
        score = heuristic_score(company, profile)
        investment_recommendations[company['Symbol']] = score

    total_score = sum(investment_recommendations.values())
    for company in investment_recommendations:
        investment_recommendations[company] = (investment_recommendations[company] / total_score) * amount

    return investment_recommendations

# Example usage
user_profile = {
    'risk_tolerance': 'moderate',
    'investment_goals': 'long_term_growth',
    'total_investment': 10000
}

company_list = ['AAPL', 'MSFT', 'GOOGL']
company_data = [get_company_data(symbol) for symbol in company_list]

recommendations = recommend_investment(user_profile['total_investment'], company_data, user_profile)
for company, amount in recommendations.items():
    print(f"Invest ${amount:.2f} in {company}")


Invest $2122.08 in AAPL
Invest $5464.16 in MSFT
Invest $2413.76 in GOOGL
